In [0]:
import numpy as np
import pandas as pd
import time
import math
#Tfidf && LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
#pickle for save and load mode
import pickle
#KDtree
from scipy import spatial
#Text prerocessing
import string
import re
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
#Increase stack limit
import sys
sys.setrecursionlimit(10000)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#DATA PART
DATA = '/content/drive/My Drive/Data'
CORPUS = '/content/drive/My Drive/Data/CORPUS'
LDAMODEL = '/content/drive/My Drive/Data/MODEL/LDAmodel'
VECTORIZEMODEL = '/content/drive/My Drive/Data/MODEL/VectorizeModel'
TFIDF = '/content/drive/My Drive/Data/TFIDF'
KDTREE = '/content/drive/My Drive/Data/KDTree'
MAINCORPUS = '/content/drive/My Drive/Data/preprocessed_copus.csv'

In [0]:
#preprocessing
redundant_word = ['code', 'tel', 'fax', 'vat', 'hscode', 'h']
def preprocessing(text):
  try:
    text = text.lower()
    text = re.sub(r'\d+', " ", text) # delete number
    text = REPLACE_BY_SPACE_RE.sub(" ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(" ", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = text.translate(text.maketrans(" ", " ", string.punctuation))
    text = word_tokenize(text)
    text = [word for word in text if not word in STOPWORDS]
    text = [word for word in text if not word in redundant_word]
    stemmer= PorterStemmer()
    # lemmatizer=WordNetLemmatizer()
    text = [stemmer.stem(word) for word in text]
    # text = [lemmatizer.lemmatize(word) for word in text]
    text = ' '.join(word for word in text)
  except:
    print('exception: ' + text)
  return text

In [0]:
#Load LDA
LDAmodel = pickle.load(open(LDAMODEL, 'rb'))

In [0]:
#Load Vectorize
VectorizeModel = pickle.load(open(VECTORIZEMODEL, 'rb'))

In [0]:
#Load main corpus
MainCorpus = pd.read_csv(MAINCORPUS)

In [0]:
#Get $topic most similar topic
def findTopic(text, topic = 5):
  text = preprocessing(text)

  topic_values = LDAmodel.transform(VectorizeModel.transform([text]))

  tops = np.argpartition(topic_values[0], -topic)[-topic:]
  tops = tops[np.argsort(topic_values[0][tops])][::-1]
  
  return text, tops

In [0]:
#Get $value_range records from topic
def findCorpus(topic_index, text, value_range = 5):
  tfidf_model = pickle.load(open(TFIDF + '/TFIDFtopic_' + str(topic_index), 'rb'))
  tfidf_vector = tfidf_model.transform([text]).todense()

  kdtree_model = pickle.load(open(KDTREE + '/KDTree_' + str(topic_index), 'rb'))
  _, top_neigbors_index = kdtree_model.query(tfidf_vector[0], k = value_range)

  corpus = pd.read_csv(CORPUS + '/topic_' + str(topic_index))
  corpus = corpus.loc[corpus['Unnamed: 0'].isin(top_neigbors_index[0])]
  
  return corpus

In [0]:
#Concat corpuses toghether and cal the distances
def findHS(corpuses, text, record):
  output_dataframe = pd.DataFrame()
  for corpus in corpuses:
    output_dataframe = output_dataframe.append(corpus)

  print(output_dataframe.head(25))

  contents = output_dataframe['content'].as_matrix()
  contents = [e for e in contents] + [text]

  distance_cal_tfidf = TfidfVectorizer()
  distance_vector = distance_cal_tfidf.fit_transform(contents).todense()

  cosine_distance = [ 1 - cosine(element.reshape(-1), distance_vector[-1].reshape(-1)) for element in distance_vector[:-1]]
  
  output_dataframe['Distance'] = cosine_distance

  return output_dataframe.sort_values(by = ['Distance'], ascending = False).head(record)

In [0]:
def mainHandler(text, topic = 5, value_range = 5, record = 10):
  # record must less than topic*value_range
  start_time = time.time()

  preprocessed_text, tops = findTopic(text, topic)
  print('text: ' + str(preprocessed_text))
  print('---------------------------------------------')
  print('tops: ' + str(tops))
  print('---------------------------------------------')

  corpuses = [findCorpus(topic_index, preprocessed_text, value_range) for topic_index in tops]
  
  print('---------------------------------------------')
  print('corpuses: ' + str(corpuses))
  print('---------------------------------------------')

  output_df = findHS(corpuses, preprocessed_text, record)

  print('---------------------------------------------')
  print('taked: ' + str(time.time() - start_time))
  print('---------------------------------------------')

  return output_df

In [0]:
text_sample = """TWO (02)x40'HC CONTAINER(S) STC\nELECTRIC WIRE\nHS CODE: 854449"""

In [0]:
mainHandler(text_sample, 5, 5)